In [ ]:
#Load packages for data wrangling
import numpy as np
import pandas as pd
import os

#Load packages for finetuning classification model and saving it
import torch
import torch.nn as nn
from simpletransformers.classification import ClassificationModel

# specify GPU
device = torch.device("cuda")

In [ ]:
from google.colab import drive #Using google colab - connecting to google drive
drive.mount("/content/drive")

#Loading training data
train_df = pd.read_csv(
    "/content/drive/My Drive/hatespeech/Train_Hate.tsv",
    sep='\t',
    quotechar='"',
    error_bad_lines=False,
    engine="python",
    encoding="UTF-8"
)

#Loading testing data
test_df = pd.read_csv(
    "/content/drive/My Drive/hatespeech/Test_Hate.tsv",
    sep='\t',
    quotechar='"',
    error_bad_lines=False,
    engine="python",
    encoding="UTF-8"
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = train_df.dropna() #Remove NAs
test_df = test_df.dropna()#Remove NAs
train_df["label"] = np.where(train_df["subtask_a"] == "NOT", 0,1) #Create binary label column
test_df["label"] = np.where(test_df["subtask_a"] == "NOT", 0,1) #Create binary label column
train_df = train_df[["tweet", "label"]] #Select relevant columns
test_df = test_df[["tweet", "label"]] #Select relevant columns

In [ ]:
# number of unique labels
n_labels = len(train_df['label'].unique())

# initialize the model with the right hyperparameters
sent_model = ClassificationModel('bert',"DJSammy/bert-base-danish-uncased_BotXO,ai",
                                 num_labels=n_labels, use_cuda=True,
                                 args={'reprocess_input_data': True, 'overwrite_output_dir': True,
                                       "num_train_epochs": 10, "max_seq_length": 128, "train_batch_size": 16,
                                       "learning_rate": 1e-5})
sent_model.train_model(train_df) #Train model

Model was saved in google drive and imported again to create a version that is executable using transformers

In [ ]:
from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained('/content/drive/My Drive/hatespeech/hater/', from_pt = True)
model.save_pretrained('/content/drive/My Drive/hatespeech/hater/')